# PyTorch Assignment: Reinforcement Learning (RL)

**[Duke Community Standard](http://integrity.duke.edu/standard.html): By typing your name below, you are certifying that you have adhered to the Duke Community Standard in completing this assignment.**

Name: Rui Yang

### Short answer

1\. One of the fundamental challenges of reinforcement learning is balancing *exploration* versus *exploitation*. What do these two terms mean, and why do they present a challenge?

Exploration involves trying out new actions and visiting new states that the agent hasn't experienced much before. The benefit of exploration is that it allows the agent to learn more about the environment, potentially discovering better strategies than it currently knows. However, the downside is that it often involves taking suboptimal actions that could lead to low rewards or even penalties.

Exploitation involves choosing the best action based on the agent's current knowledge. If the agent has learned a good policy, exploitation will often lead to high rewards. However, the downside is that if the agent's knowledge is incomplete or incorrect, exploitation could lead to suboptimal actions. Also, by only exploiting known information, the agent might miss out on opportunities to discover even better strategies.

The challenge of balancing exploration and exploitation, often referred to as the exploration-exploitation dilemma, arises because it's usually not possible to both explore and exploit at the same time. If an agent spends too much time exploring, it might miss out on rewards it could have gained by exploiting what it has already learned. On the other hand, if it spends too much time exploiting, it might miss out on learning better strategies.

In practice, reinforcement learning algorithms often start with a high rate of exploration to learn about the environment, and gradually shift towards exploitation as they gain more knowledge. However, finding the right balance is a complex problem that depends on many factors, including the specifics of the environment, the learning algorithm, and the agent's current knowledge.

2\. Another fundamental reinforcement learning challenge is what is known as the *credit assignment problem*, especially when rewards are sparse. What do we mean by the phrase, and why does it make learning especially difficult?

The credit assignment problem in reinforcement learning refers to the challenge of determining which actions in a sequence (or episode) are responsible for the eventual outcome or reward. This problem becomes especially difficult when rewards are sparse, delayed, or both.

In many reinforcement learning scenarios, an agent makes a sequence of decisions (actions), and the reward comes at the end of a series of actions, or intermittently. In such cases, it can be difficult to understand which actions were beneficial and contributed to the reward and which did not. This is the credit assignment problem: figuring out how to "credit" or "blame" each action for the part it played in achieving the final outcome.

When rewards are sparse (that is, few and far between), the credit assignment problem becomes even more difficult. If an agent only receives feedback once every thousand actions, for instance, it can be extremely hard to determine which of those thousand actions were beneficial and which were not.

This problem can make learning especially difficult for several reasons:

Delayed Rewards: In many tasks, a reward is only obtained after a series of correct actions. It's difficult to figure out which actions in the sequence were important for getting the reward, especially if the reward is delayed.

Noise: If there's a lot of noise in the environment or in the rewards, it can be hard to tell whether a given action was good or bad.

Exploration: If rewards are sparse, it can take a long time to explore the environment sufficiently to get a good estimate of the value of each action.

Solving the credit assignment problem is a major focus of many reinforcement learning algorithms. Techniques like Temporal Difference Learning (TD Learning), Monte Carlo methods, and the use of eligibility traces are all methods designed to help with credit assignment.

### Deep SARSA Cart Pole

[SARSA (state-action-reward-state-action)](https://en.wikipedia.org/wiki/State–action–reward–state–action) is another Q value algorithm that resembles Q-learning quite closely:

Q-learning update rule:
\begin{equation}
Q_\pi (s_t, a_t) \leftarrow (1 - \alpha) \cdot Q_\pi(s_t, a_t) + \alpha \cdot \big(r_t + \gamma \max_a Q_\pi(s_{t+1}, a)\big)
\end{equation}

SARSA update rule:
\begin{equation}
Q_\pi (s_t, a_t) \leftarrow (1 - \alpha) \cdot Q_\pi(s_t, a_t) + \alpha \cdot \big(r_t + \gamma Q_\pi(s_{t+1}, a_{t+1})\big)
\end{equation}

Unlike Q-learning, which is considered an *off-policy* network, SARSA is an *on-policy* algorithm. 
When Q-learning calculates the estimated future reward, it must "guess" the future, starting with the next action the agent will take. In Q-learning, we assume the agent will take the best possible action: $\max_a Q_\pi(s_{t+1}, a)$. SARSA, on the other hand, uses the action that was actually taken next in the episode we are learning from: $Q_\pi(s_{t+1}, a_{t+1})$. In other words, SARSA learns from the next action he actually took (on policy), as opposed to what the max possible Q value for the next state was (off policy).

Build an RL agent that uses SARSA to solve the Cart Pole problem. 

*Hint: You can and should reuse the Q-Learning agent we went over earlier. In fact, if you know what you're doing, it's possible to finish this assignment in about 30 seconds.*

In [3]:
### YOUR CODE HERE ###
from collections import deque
import random
import math

import gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


class DQN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 24)
        self.fc2 = nn.Linear(24, 48)
        self.fc3 = nn.Linear(48, 2)

    def forward(self, x):        
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x        
    

class DQNCartPoleSolver:
    def __init__(self, n_episodes=1000, n_win_ticks=195, max_env_steps=None, gamma=1.0, epsilon=1.0, epsilon_min=0.01, epsilon_log_decay=0.995, alpha=0.01, alpha_decay=0.01, batch_size=64, monitor=False, quiet=False):
        self.memory = deque(maxlen=100000)
        self.env = gym.make('CartPole-v0')
        if monitor: self.env = gym.wrappers.Monitor(self.env, '../data/cartpole-1', force=True)
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_log_decay
        self.alpha = alpha
        self.alpha_decay = alpha_decay
        self.n_episodes = n_episodes
        self.n_win_ticks = n_win_ticks
        self.batch_size = batch_size
        self.quiet = quiet
        if max_env_steps is not None: self.env._max_episode_steps = max_env_steps

        # Init model
        self.dqn = DQN()
        self.criterion = torch.nn.MSELoss()
        self.opt = torch.optim.Adam(self.dqn.parameters(), lr=0.01)

    def get_epsilon(self, t):
        return max(self.epsilon_min, min(self.epsilon, 1.0 - math.log10((t + 1) * self.epsilon_decay)))

    def preprocess_state(self, state):
        return torch.tensor(np.reshape(state, [1, 4]), dtype=torch.float32) 
    
    def choose_action(self, state, epsilon):
        if (np.random.random() <= epsilon):
            return self.env.action_space.sample() 
        else:
            with torch.no_grad():
                return torch.argmax(self.dqn(state)).numpy()

    def remember(self, state, action, reward, next_state, done):
        reward = torch.tensor(reward)
        self.memory.append((state, action, reward, next_state, done))
    
    def replay(self, batch_size):
        y_batch, y_target_batch = [], []
        minibatch = random.sample(self.memory, min(len(self.memory), batch_size))
        for state, action, reward, next_state, done in minibatch:
            y = self.dqn(state)
            y_target = y.clone().detach()
            with torch.no_grad():
                y_target[0][action] = reward if done else reward + self.gamma * torch.max(self.dqn(next_state)[0])
            y_batch.append(y[0])
            y_target_batch.append(y_target[0])
        
        y_batch = torch.cat(y_batch)
        y_target_batch = torch.cat(y_target_batch)
        
        self.opt.zero_grad()
        loss = self.criterion(y_batch, y_target_batch)
        loss.backward()
        self.opt.step()        
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def run(self):
        scores = deque(maxlen=100)

        for e in range(self.n_episodes):
            state = self.preprocess_state(self.env.reset())
            action = self.choose_action(state, self.get_epsilon(e))  # Take initial action
            done = False
            i = 0
            while not done:
                next_state, reward, done, _ = self.env.step(action)
                next_state = self.preprocess_state(next_state)
                next_action = self.choose_action(next_state, self.get_epsilon(e))  # Choose next action
                self.remember(state, action, reward, next_state, done)
                self.replay(self.batch_size)  # Replay function now uses SARSA update
                state = next_state
                action = next_action  # Next action becomes current action
                i += 1

            scores.append(i)
            mean_score = np.mean(scores)
            if mean_score >= self.n_win_ticks and e >= 100:
                if not self.quiet: print('Ran {} episodes. Solved after {} trials ✔'.format(e, e - 100))
                return e - 100
            if e % 100 == 0 and not self.quiet:
                print('[Episode {}] - Mean survival time over last 100 episodes was {} ticks.'.format(e, mean_score))

            self.replay(self.batch_size)
        
        if not self.quiet: print('Did not solve after {} episodes 😞'.format(e))
        return e

if __name__ == '__main__':
    agent = DQNCartPoleSolver()
    agent.run()
    agent.env.close()


[Episode 0] - Mean survival time over last 100 episodes was 16.0 ticks.
[Episode 100] - Mean survival time over last 100 episodes was 13.47 ticks.
[Episode 200] - Mean survival time over last 100 episodes was 39.56 ticks.
[Episode 300] - Mean survival time over last 100 episodes was 68.11 ticks.
[Episode 400] - Mean survival time over last 100 episodes was 159.92 ticks.
Ran 436 episodes. Solved after 336 trials ✔
